In [1]:
# numpy 로드하여 np로 사용
# pandas 로드하여 pd로 사용
# matplotlib.pyplot 로드하여 plt로 사용
# seaborn 로드하여 sns로 사용
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

In [15]:
import pandas as pd

# 상대경로 기준: check.ipynb → code/visualization/ → ../ → ../ → data/
df = pd.read_csv("../../data/Mcdelivery_menu_prices_Kacl.csv")

# 데이터 확인
print(df.head())
print(df.columns)

      카테고리                메뉴     가격  칼로리(Kcal)
0  버거 & 세트        더블 1955 버거  10300        807
1  버거 & 세트   더블 맥스파이시 상하이 버거   9700        759
2  버거 & 세트  맥스파이시 상하이 투움바 버거   8100        489
3  버거 & 세트       더블 쿼터파운더 치즈   8200        770
4  버거 & 세트          쿼터파운더 치즈   6300        536
Index(['카테고리', '메뉴', '가격', '칼로리(Kcal)'], dtype='object')


In [23]:
import pandas as pd

# ✅ 데이터 불러오기
df = pd.read_csv("../../data/Mcdelivery_menu_prices_Kacl.csv")

# ✅ 데이터 전처리
df['가격'] = df['가격'].astype(str).str.replace(",", "").astype(int)
df['칼로리(Kcal)'] = df['칼로리(Kcal)'].astype(str).str.replace(r"[^\d.]", "", regex=True)
df['칼로리(Kcal)'] = pd.to_numeric(df['칼로리(Kcal)'], errors='coerce')

# ✅ NaN 제거
df = df.dropna(subset=['칼로리(Kcal)'])

# ✅ cost_per_cal 계산
df['cost_per_cal'] = df['가격'] / df['칼로리(Kcal)']

# ✅ "버거 & 세트" 카테고리만 필터링
burger_df = df[df['카테고리'] == "버거 & 세트"].copy()

# ✅ 가장 가성비 좋은 메뉴 찾기
best_idx = burger_df['cost_per_cal'].idxmin()
best = burger_df.loc[best_idx]

# ✅ 특정 메뉴와 비교 예시 (예: 빅맥)
selected_name = "더블 맥스파이시"
target = burger_df[burger_df['메뉴'].str.contains(selected_name, na=False)]

if not target.empty:
    t = target.iloc[0]
    print(f"🔍 '{t['메뉴']}' 칼로리당 가격: {t['cost_per_cal']:.2f} 원/kcal")
else:
    print(f"⚠️ '{selected_name}' 메뉴는 버거 & 세트에서 찾을 수 없습니다.")

# ✅ 가성비 1등 메뉴 출력
print(f"🏆 가장 가성비 좋은 메뉴 (버거 & 세트): {best['메뉴']}")
print(f"   가격: {best['가격']}원 / 칼로리: {best['칼로리(Kcal)']} kcal")
print(f"   칼로리당 가격: {best['cost_per_cal']:.2f} 원/kcal")

🔍 '더블 맥스파이시 상하이 버거' 칼로리당 가격: 12.78 원/kcal
🏆 가장 가성비 좋은 메뉴 (버거 & 세트): 맥치킨 모짜렐라
   가격: 5800원 / 칼로리: 743 kcal
   칼로리당 가격: 7.81 원/kcal


In [26]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

# 🔐 .env에서 Nutritionix API Key 불러오기
load_dotenv()
APP_ID = os.getenv("NUTRITIONIX_APP_ID")
APP_KEY = os.getenv("NUTRITIONIX_APP_KEY")

# ✅ 사용자 정보
USER_PROFILE = {
    "gender": "male",
    "weight_kg": 70,
    "height_cm": 175,
    "age": 25
}

# ✅ 데이터 불러오기
df = pd.read_csv("../../data/Mcdelivery_menu_prices_Kacl.csv")

# ✅ 전처리
df['가격'] = df['가격'].astype(str).str.replace(",", "").astype(int)
df['칼로리(Kcal)'] = df['칼로리(Kcal)'].astype(str).str.replace(r"[^\d.]", "", regex=True)
df['칼로리(Kcal)'] = pd.to_numeric(df['칼로리(Kcal)'], errors='coerce')
df = df.dropna(subset=['칼로리(Kcal)'])

df['cost_per_cal'] = df['가격'] / df['칼로리(Kcal)']
burger_df = df[df['카테고리'] == "버거 & 세트"].copy()

# ✅ 사용자 선택 메뉴
selected_name = "더블 맥스파이시"
target = burger_df[burger_df['메뉴'].str.contains(selected_name, na=False)]

if not target.empty:
    t = target.iloc[0]
    menu_name = t['메뉴']
    kcal = t['칼로리(Kcal)']
    print(f"🍔 선택한 메뉴: {menu_name} / 칼로리: {kcal:.0f} kcal")

    # ✅ 걷기 30분 기준 칼로리 소모량 요청
    query = {
        "query": "walking 30 minutes",  # ← 영어로!
        **USER_PROFILE
    }

    headers = {
        "x-app-id": APP_ID,
        "x-app-key": APP_KEY,
        "Content-Type": "application/json"
    }

    url = "https://trackapi.nutritionix.com/v2/natural/exercise"
    response = requests.post(url, json=query, headers=headers)

    if response.status_code == 200:
        data = response.json()
        walking = data['exercises'][0]
        burn_per_min = walking['nf_calories'] / walking['duration_min']
        required_time = kcal / burn_per_min

        print(f"🚶 1분 걷기로 약 {burn_per_min:.2f} kcal 소모")
        print(f"🔥 '{menu_name}' 섭취 칼로리를 모두 소모하려면 약 {required_time:.1f}분 걷기 필요!")
    else:
        print("❌ 운동 칼로리 계산 실패:", response.status_code)
        print(response.text)
else:
    print(f"⚠️ '{selected_name}' 메뉴는 '버거 & 세트'에서 찾을 수 없습니다.")


🍔 선택한 메뉴: 더블 맥스파이시 상하이 버거 / 칼로리: 759 kcal
🚶 1분 걷기로 약 4.08 kcal 소모
🔥 '더블 맥스파이시 상하이 버거' 섭취 칼로리를 모두 소모하려면 약 185.9분 걷기 필요!
